In [3]:


import sqlite3



In [15]:
! mkdir -p databases
DB_PATH = "databases/test21_2.db"
connection = sqlite3.connect(DB_PATH)


Используя базу данных с предыдущего занятия, надо написать класс (или набор функций) для работы с ней. Должны быть реализованы возможности:

    Вывести список книг. 
   
    Вывести список читателей. 
    Добавить книгу.
    Добавить читателя. 
    Выдать книгу читателю.
    Принять книгу.




Books (id, author, title, publish_year)
Readers (id, name)
Records (reader_id, book_id, taking_date, returning_date)



Создаем таблицы 

In [16]:
c = connection.cursor()

# Create table
c.execute('''CREATE TABLE Books (id REAL, title TEXT, author TEXT, publish_year REAL); ''')


In [17]:
c.execute(''' 
    INSERT INTO Books VALUES 
        (10, 'Fullmetal Alchemist', 'Hiromu Arakawa', 2010), 
        (8, 'Dragon Ball',  'Akira Toriyama', 1995), 
        (6,'Inuyasha',  'Rumiko Takahashi' , 2008), 
        (2, 'Naruto',  'Masashi Kishimoto' , 2014),
        (5, 'Sailor Moon' , 'Naoko Takeuchi', 1997),
        (7, 'Death Note', 'Tsugumi Ohba and Takeshi Obata', 2006);
''')

In [18]:
c.execute('''CREATE TABLE Readers (id REAL, name TEXT ); ''')


c.execute(''' 
    INSERT INTO Readers VALUES 
        (1, 'Nikolay Biba'), 
        (2, 'Aleksandra Boba'); 

''')

In [19]:
c.execute('''CREATE TABLE Records (reader_id REAL, book_id REAL, taking_date REAL, returning_date REAL); ''')

c.execute('''
    INSERT INTO Records VALUES
        (1, 10, 13.03, 15.05);
''')


In [20]:
connection.commit()
connection.close()

In [21]:
connection = sqlite3.connect(DB_PATH)
c = connection.cursor()

c.execute("SELECT * FROM Books WHERE author='Naoko Takeuchi';")

print(c.fetchone())


(5.0, 'Sailor Moon', 'Naoko Takeuchi', 1997.0)


In [22]:
connection.commit()
connection.close()

#### Вывести список книг.

In [23]:
def look_at_books():
    connection = sqlite3.connect(DB_PATH)
    cursor = connection.cursor()

    for row in cursor.execute("SELECT * FROM Books ORDER BY id;"):
        print(row)

    connection.close()
    return

look_at_books()

(2.0, 'Naruto', 'Masashi Kishimoto', 2014.0)
(5.0, 'Sailor Moon', 'Naoko Takeuchi', 1997.0)
(6.0, 'Inuyasha', 'Rumiko Takahashi', 2008.0)
(7.0, 'Death Note', 'Tsugumi Ohba and Takeshi Obata', 2006.0)
(8.0, 'Dragon Ball', 'Akira Toriyama', 1995.0)
(10.0, 'Fullmetal Alchemist', 'Hiromu Arakawa', 2010.0)


#### Вывести список читателей.

In [24]:
def look_at_readers():
    connection = sqlite3.connect(DB_PATH)
    cursor = connection.cursor()

    for row in cursor.execute("SELECT * FROM Readers ORDER BY id;"):
        print(row)

    connection.close()

look_at_readers()

(1.0, 'Nikolay Biba')
(2.0, 'Aleksandra Boba')


#### Добавить книгу.

In [25]:
def add_book(book):
    
    connection = sqlite3.connect(DB_PATH)
    cursor = connection.cursor()

    connection.executemany(
        "INSERT INTO Books VALUES (?, ?, ?, ?)",
        book
    )
    connection.commit()
    connection.close()
    return 

book = [(9, 'Soul Eater', 'Atsushi Ohkubo', 2013)]
add_book(book)


Проверим что добавилось

In [26]:
look_at_books()

(2.0, 'Naruto', 'Masashi Kishimoto', 2014.0)
(5.0, 'Sailor Moon', 'Naoko Takeuchi', 1997.0)
(6.0, 'Inuyasha', 'Rumiko Takahashi', 2008.0)
(7.0, 'Death Note', 'Tsugumi Ohba and Takeshi Obata', 2006.0)
(8.0, 'Dragon Ball', 'Akira Toriyama', 1995.0)
(9.0, 'Soul Eater', 'Atsushi Ohkubo', 2013.0)
(10.0, 'Fullmetal Alchemist', 'Hiromu Arakawa', 2010.0)


#### Добавить читателя.

In [27]:
def add_reader(reader):
    
    connection = sqlite3.connect(DB_PATH)
    cursor = connection.cursor()

    connection.executemany(
        "INSERT INTO Readers VALUES (?, ?)",
        reader
    )
    connection.commit()
    connection.close()
    return

reader = [(3, 'Vasiliy Alexeev')]
add_reader(reader)
look_at_readers()

(1.0, 'Nikolay Biba')
(2.0, 'Aleksandra Boba')
(3.0, 'Vasiliy Alexeev')


#### Выдать книгу читателю.


In [29]:

# Records (reader_id, book_id, taking_date, returning_date)

def give_book(record):
    connection = sqlite3.connect(DB_PATH)
    cursor = connection.cursor()

    connection.execute("INSERT OR IGNORE INTO Records VALUES(?, ?, ?, ?)", record)
    connection.commit()

    connection.close()
    return

In [31]:


record = (2.0, 2.0, 13.03, 13.04)

print("до выдачи книги")
connection = sqlite3.connect(DB_PATH)
cursor = connection.cursor()

for row in cursor.execute("SELECT * FROM Records ORDER BY taking_date;"):
        print(row)

connection.close()

give_book(record)
print("после выдачи книги")
connection = sqlite3.connect(DB_PATH)
cursor = connection.cursor()

for row in cursor.execute("SELECT * FROM Records ORDER BY taking_date;"):
        print(row)

connection.close()

до выдачи книги
(1.0, 10.0, 13.03, 15.05)
после выдачи книги
(1.0, 10.0, 13.03, 15.05)
(2.0, 2.0, 13.03, 13.04)


#### Принять книгу.

Хотим, зная, что за книга и читатель, сделать новый record с той же датой, когда человек брал, но с новой, когда реально отдал

In [64]:
import numpy as np

In [78]:
def accept_book(reader_id, book_id, accept_date):

    connection = sqlite3.connect(DB_PATH)

    cursor = connection.cursor()
    
    for row in cursor.execute("SELECT * FROM Records WHERE reader_id = ? AND book_id = ?;", (reader_id, book_id)):
        record = row

    new_record = tuple(np.insert(np.array(record), [3] ,accept_date)[:4])

    cursor.execute("INSERT OR IGNORE INTO Records VALUES(?, ?, ?, ?)", new_record)
    
    connection.commit()
    connection.close()

    return



In [79]:
reader_id, book_id = 1, 10
accept_date = 15.11
accept_book(reader_id, book_id, accept_date)

In [80]:
connection = sqlite3.connect(DB_PATH)
cursor = connection.cursor()

for row in cursor.execute("SELECT * FROM Records ORDER BY taking_date;"):
        print(row)

connection.close()


(1.0, 10.0, 13.03, 14.05)
(2.0, 2.0, 13.03, 13.04)
(1.0, 10.0, 13.03, 15.11)
